**Food Delivery Analysis**


**Project Objective**


The objective of this analysis is to evaluate the operational efficiency, customer experience, and revenue performance of a food delivery platform using transactional, delivery, and rating data.
The insights aim to identify growth opportunities, operational bottlenecks, and revenue leakage across cities.

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect(':memory:')

pd.read_csv('fact_orders.csv').to_sql('fact_orders', conn, index=False)
pd.read_csv('dim_customer.csv').to_sql('dim_customer', conn, index=False)
pd.read_csv('dim_restaurant.csv').to_sql('dim_restaurant', conn, index=False)
pd.read_csv('dim_delivery_partner_.csv').to_sql('dim_delivery_partner_', conn, index=False)
pd.read_csv('fact_delivery_performance.csv').to_sql('fact_delivery_performance', conn, index=False)
pd.read_csv('fact_ratings.csv').to_sql('fact_ratings', conn, index=False)
pd.read_csv('fact_order_items.csv').to_sql('fact_order_items', conn, index=False)
pd.read_csv('dim_menu_item.csv').to_sql('dim_menu_item', conn, index=False)

print(" All tables loaded successfully")

 All tables loaded successfully


**Supply Analysis – Restaurant & Delivery Partner Distribution**

1.   City‑Level Order
2.   Cuisine Distribution
3.   Restaurant Distribution



In [ ]:
query = """
SELECT
  city,
  COUNT(DISTINCT restaurant_id) AS total_restaurants
FROM dim_restaurant
WHERE LOWER(is_active) IN ('yes','true','1','y')
GROUP BY city


ORDER BY total_restaurants DESC;
"""
pd.read_sql(query, conn)


,city,total_restaurants
0,Bengaluru,4457
1,Mumbai,2720
2,Delhi,2685
3,Hyderabad,1826
4,Chennai,1810
5,Kolkata,1648
6,Ahmedabad,1463
7,Pune,1441


In [ ]:
pd.read_sql("""
SELECT
  cuisine_type,
  COUNT(DISTINCT restaurant_id) AS restaurant_count
FROM dim_restaurant
WHERE LOWER(is_active) IN ('yes','true','1','y')
GROUP BY cuisine_type
ORDER BY restaurant_count DESC;

""", conn)


,cuisine_type,restaurant_count
0,North Indian,3578
1,South Indian,2720
2,Chinese,2699
3,Biryani,2697
4,Pizza,2218
5,Fast Food,1788
6,Desserts,1458
7,Healthy,892


In [ ]:
pd.read_sql("""
SELECT
  city,
  COUNT(DISTINCT delivery_partner_id) AS total_delivery_partners
FROM dim_delivery_partner_
WHERE LOWER(is_active) IN ('yes','true','1','y')
GROUP BY city
ORDER BY total_delivery_partners DESC;

""", conn)


,city,total_delivery_partners
0,Bengaluru,3334
1,Mumbai,1979
2,Delhi,1955
3,Chennai,1378
4,Hyderabad,1275
5,Kolkata,1200
6,Ahmedabad,1068
7,Pune,1046


In [ ]:
pd.read_sql("""
SELECT
  r.city,
  COUNT(DISTINCT r.restaurant_id) AS restaurants,
  COUNT(DISTINCT d.delivery_partner_id) AS delivery_partners
FROM dim_restaurant r
LEFT JOIN dim_delivery_partner_ d
  ON r.city = d.city
  AND LOWER(TRIM(d.is_active)) IN ('yes','true','1','y')
WHERE LOWER(TRIM(r.is_active)) IN ('yes','true','1','y')
GROUP BY r.city;
""", conn)


,city,restaurants,delivery_partners
0,Ahmedabad,1463,1068
1,Bengaluru,4457,3334
2,Chennai,1810,1378
3,Delhi,2685,1955
4,Hyderabad,1826,1275
5,Kolkata,1648,1200
6,Mumbai,2720,1979
7,Pune,1441,1046


**Pricing & Order Performance**



1.   Overall Order Performance
2.   City‑Level Revenue & AOV
3.   Cuisine‑Level AOV





In [ ]:
pd.read_sql("""
SELECT
  is_cancelled,
  COUNT(*) AS cnt
FROM fact_orders
GROUP BY is_cancelled;
""", conn)


,is_cancelled,cnt
0,N,138054
1,Y,11112


In [ ]:
pd.read_sql("""
SELECT
  COUNT(order_id) AS total_orders,
  ROUND(AVG(total_amount), 2) AS avg_order_value
FROM fact_orders
WHERE is_cancelled = 'N';
""", conn)


,total_orders,avg_order_value
0,138054,351.75


Average Order Value = Total Revenue from successful orders ÷ Number of successful orders

In [ ]:
pd.read_sql("""
SELECT
  SUM(total_amount) AS total_revenue,
  COUNT(order_id) AS total_orders,
  ROUND(SUM(total_amount) * 1.0 / COUNT(order_id), 2) AS calculated_aov
FROM fact_orders
WHERE is_cancelled = 'N';
""",conn)


,total_revenue,total_orders,calculated_aov
0,48561115.56,138054,351.75


In [ ]:
pd.read_sql("""
SELECT
  r.city,
  SUM(total_amount) AS total_revenue,
  COUNT(o.order_id) AS total_orders,
  ROUND(AVG(o.total_amount), 2) AS avg_order_value
FROM fact_orders o
JOIN dim_restaurant r
  ON o.restaurant_id = r.restaurant_id
WHERE is_cancelled = 'N'
GROUP BY r.city
ORDER BY avg_order_value DESC;

""",conn)

,city,total_revenue,total_orders,avg_order_value
0,Ahmedabad,4002341.50,11322,353.50
1,Mumbai,7207584.47,20437,352.67
2,Hyderabad,4936031.42,13997,352.65
3,Delhi,7224132.36,20515,352.14
4,Chennai,4875450.76,13878,351.31
5,Bengaluru,11989687.97,34154,351.05
6,Pune,3877140.29,11055,350.71
7,Kolkata,4448746.79,12696,350.41


In [ ]:
pd.read_sql("""
SELECT
  r.restaurant_name,
  r.city,
  SUM(total_amount) AS total_revenue,
  COUNT(o.order_id) AS total_orders,
  ROUND(AVG(o.total_amount), 2) AS avg_order_value
FROM fact_orders o
JOIN dim_restaurant r
  ON o.restaurant_id = r.restaurant_id
WHERE is_cancelled = 'N'
GROUP BY r.restaurant_name, r.city
HAVING COUNT(o.order_id) >= 20
ORDER BY avg_order_value DESC;

""",conn)

,restaurant_name,city,total_revenue,total_orders,avg_order_value
0,Sri Tandoor Stop,Kolkata,9341.17,22,424.60
1,Tandoori Cafe Point,Bengaluru,10165.24,24,423.55
2,Fresh Kitchen Clouds,Delhi,9258.14,22,420.82
3,Thindi Mane Wraps Clouds,Bengaluru,8343.39,20,417.17
4,Flavours of Kitchen Bhojanalay,Mumbai,8221.57,20,411.08
...,...,...,...,...,...
798,Sri Darshini Stop,Chennai,5850.54,20,292.53
799,Tandoori Sweets Zone,Bengaluru,8412.01,29,290.07
800,Hot & Crispy Biryani Mahal,Bengaluru,6909.83,24,287.91
801,Urban Pizza Central,Bengaluru,5555.11,20,277.76


In [ ]:
pd.read_sql("""
SELECT
  r.cuisine_type,
  SUM(total_amount) AS total_revenue,
  COUNT(o.order_id) AS total_orders,
  ROUND(AVG(o.total_amount), 2) AS avg_order_value
FROM fact_orders o
JOIN dim_restaurant r
  ON o.restaurant_id = r.restaurant_id
WHERE is_cancelled = 'N'
GROUP BY r.cuisine_type
ORDER BY avg_order_value DESC;

""",conn)

,cuisine_type,total_revenue,total_orders,avg_order_value
0,Desserts,3950418.15,11196,352.84
1,South Indian,7321239.54,20781,352.30
2,North Indian,9594578.03,27241,352.21
3,Pizza,5997000.79,17052,351.69
4,Biryani,7341619.20,20884,351.54
5,Healthy,2457807.17,6999,351.17
6,Chinese,7173284.92,20431,351.10
7,Fast Food,4725167.76,13470,350.79


**Order Value Segmentation**

Most orders fall in the ₹200–₹400 range, with very few high‑value (₹600+) orders.

In [ ]:
pd.read_sql("""
SELECT
  CASE
    WHEN total_amount < 200 THEN '< 200'
    WHEN total_amount BETWEEN 200 AND 400 THEN '200–400'
    WHEN total_amount BETWEEN 401 AND 600 THEN '401–600'
    ELSE '600+'
  END AS order_value_bucket,
  COUNT(order_id) AS orders
FROM fact_orders
WHERE is_cancelled = 'N'
GROUP BY order_value_bucket
ORDER BY orders DESC;

""",conn)

,order_value_bucket,orders
0,200–400,90698
1,401–600,33900
2,< 200,7805
3,600+,5651


 **Operational Efficiency**



1.   Cancellation Rate
2.   Delivery Time & SLA Performance



In [ ]:
pd.read_sql("""
SELECT
  COUNT(order_id) AS total_orders,
  SUM(CASE WHEN is_cancelled = 'Y' THEN 1 ELSE 0 END) AS cancelled_orders,
  ROUND(
    SUM(CASE WHEN is_cancelled = 'Y' THEN 1 ELSE 0 END) * 100.0
    / COUNT(order_id),
    2
  ) AS cancellation_rate_pct
FROM fact_orders;

""",conn)

,total_orders,cancelled_orders,cancellation_rate_pct
0,149166,11112,7.45


In [ ]:
pd.read_sql("""
SELECT
  r.city,
  COUNT(o.order_id) AS total_orders,
  SUM(CASE WHEN o.is_cancelled = 'Y' THEN 1 ELSE 0 END) AS cancelled_orders,
  ROUND(
    SUM(CASE WHEN o.is_cancelled = 'Y' THEN 1 ELSE 0 END) * 100.0
    / COUNT(o.order_id),
    2
  ) AS cancellation_rate_pct
FROM fact_orders o
JOIN dim_restaurant r
  ON o.restaurant_id = r.restaurant_id
GROUP BY r.city
ORDER BY cancellation_rate_pct DESC;

""",conn)

,city,total_orders,cancelled_orders,cancellation_rate_pct
0,Ahmedabad,12271,949,7.73
1,Hyderabad,15135,1138,7.52
2,Bengaluru,36919,2765,7.49
3,Chennai,15000,1122,7.48
4,Mumbai,22073,1636,7.41
5,Pune,11934,879,7.37
6,Delhi,22138,1623,7.33
7,Kolkata,13696,1000,7.30


In [ ]:
pd.read_sql("""
SELECT
  ROUND(AVG(actual_delivery_time_mins), 2) AS avg_delivery_time_mins
FROM fact_delivery_performance;

""",conn)

,avg_delivery_time_mins
0,44.4


In [ ]:
pd.read_sql("""
SELECT
  COUNT(order_id) AS total_deliveries,
  SUM(
    CASE
      WHEN actual_delivery_time_mins > expected_delivery_time_mins THEN 1
      ELSE 0
    END
  ) AS sla_breaches,
  ROUND(
    SUM(
      CASE
        WHEN actual_delivery_time_mins > expected_delivery_time_mins THEN 1
        ELSE 0
      END
    ) * 100.0 / COUNT(order_id),
    2
  ) AS sla_breach_pct
FROM fact_delivery_performance;

""",conn)

,total_deliveries,sla_breaches,sla_breach_pct
0,149166,95226,63.84


In [ ]:
pd.read_sql("""
SELECT
  r.city,
  COUNT(d.order_id) AS total_orders,
  SUM(
    CASE
      WHEN d.actual_delivery_time_mins > d.expected_delivery_time_mins THEN 1
      ELSE 0
    END
  ) AS sla_breaches,
  ROUND(
    SUM(
      CASE
        WHEN d.actual_delivery_time_mins > d.expected_delivery_time_mins THEN 1
        ELSE 0
      END
    ) * 100.0 / COUNT(d.order_id),
    2
  ) AS sla_breach_pct
FROM fact_delivery_performance d
JOIN fact_orders o
  ON d.order_id = o.order_id
JOIN dim_restaurant r
  ON o.restaurant_id = r.restaurant_id
GROUP BY r.city
ORDER BY sla_breach_pct DESC;

""",conn)

,city,total_orders,sla_breaches,sla_breach_pct
0,Pune,11934,7674,64.30
1,Ahmedabad,12271,7880,64.22
2,Mumbai,22073,14139,64.06
3,Delhi,22138,14167,63.99
4,Kolkata,13696,8729,63.73
5,Hyderabad,15135,9645,63.73
6,Bengaluru,36919,23523,63.72
7,Chennai,15000,9469,63.13


In [ ]:
pd.read_sql("""
SELECT
  dp.partner_name,
  COUNT(o.order_id) AS total_orders,
  ROUND(AVG(d.actual_delivery_time_mins), 2) AS avg_delivery_time
FROM fact_orders o
JOIN fact_delivery_performance d
  ON o.order_id = d.order_id
JOIN dim_delivery_partner_ dp
  ON o.delivery_partner_id = dp.delivery_partner_id
GROUP BY dp.partner_name
ORDER BY avg_delivery_time DESC;

""",conn)

,partner_name,total_orders,avg_delivery_time
0,Rahul V,166,46.99
1,Ananya T,357,46.42
2,Shreya S,307,46.19
3,Pooja A,315,46.14
4,Suresh S,285,46.06
...,...,...,...
515,Ravi D,278,42.45
516,Sneha D,163,42.39
517,Priya I,322,42.39
518,Pooja J,249,42.15


**Customer Experience & Revenue Impact Analysis**

1.   Overall Ratings
2.   Estimated Revenue Loss Due to Cancellations



In [ ]:
pd.read_sql("""
SELECT
  COUNT(order_id) AS total_ratings,
  ROUND(AVG(rating), 2) AS avg_rating
FROM fact_ratings;
""",conn)

,total_ratings,avg_rating
0,68825,4.05


In [ ]:
pd.read_sql("""
SELECT
  r.city,
  COUNT(fr.order_id) AS total_orders,
  ROUND(AVG(fr.rating), 2) AS avg_rating
FROM fact_ratings fr
JOIN fact_orders o
  ON fr.order_id = o.order_id
JOIN dim_restaurant r
  ON o.restaurant_id = r.restaurant_id
GROUP BY r.city
ORDER BY avg_rating ASC;


""",conn)

,city,total_orders,avg_rating
0,Pune,5535,4.03
1,Hyderabad,6884,4.04
2,Bengaluru,17072,4.05
3,Delhi,10318,4.05
4,Chennai,6970,4.06
5,Mumbai,10079,4.06
6,Ahmedabad,5559,4.07
7,Kolkata,6408,4.07


In [ ]:
pd.read_sql("""
SELECT
  CASE
    WHEN d.actual_delivery_time_mins <= 30 THEN '0–30 mins'
    WHEN d.actual_delivery_time_mins <= 45 THEN '31–45 mins'
    WHEN d.actual_delivery_time_mins <= 60 THEN '46–60 mins'
    ELSE '60+ mins'
  END AS delivery_time_bucket,
  COUNT(fr.order_id) AS total_orders,
  ROUND(AVG(fr.rating), 2) AS avg_rating
FROM fact_ratings fr
JOIN fact_delivery_performance d
  ON fr.order_id = d.order_id
GROUP BY delivery_time_bucket
ORDER BY delivery_time_bucket;

""",conn)

,delivery_time_bucket,total_orders,avg_rating
0,0–30 mins,6966,4.50
1,31–45 mins,36963,4.36
2,46–60 mins,17586,3.88
3,60+ mins,7310,2.49


In [ ]:
pd.read_sql("""
SELECT
  COUNT(order_id) * (
    SELECT AVG(total_amount)
    FROM fact_orders
    WHERE LOWER(is_cancelled) IN ('n','no','false','0')
  ) AS estimated_revenue_lost
FROM fact_orders
WHERE LOWER(is_cancelled) IN ('y','yes','true','1');

""",conn)

,estimated_revenue_lost
0,3.908696e+06


In [ ]:
pd.read_sql("""
SELECT
  r.city,
  COUNT(o.order_id) AS cancelled_orders,
  ROUND(
    COUNT(o.order_id) * (
      SELECT AVG(total_amount)
      FROM fact_orders
      WHERE LOWER(is_cancelled) IN ('n','no','false','0')
    ),
    2
  ) AS estimated_revenue_lost
FROM fact_orders o
JOIN dim_restaurant r
  ON o.restaurant_id = r.restaurant_id
WHERE LOWER(o.is_cancelled) IN ('y','yes','true','1')
GROUP BY r.city
ORDER BY estimated_revenue_lost DESC;

""",conn)

,city,cancelled_orders,estimated_revenue_lost
0,Bengaluru,2765,972601.19
1,Mumbai,1636,575470.36
2,Delhi,1623,570897.55
3,Hyderabad,1138,400296.62
4,Chennai,1122,394668.55
5,Kolkata,1000,351754.50
6,Ahmedabad,949,333815.02
7,Pune,879,309192.20
